# FINM 36700 HW 5
### HW Group B 6

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

## Data

In [2]:
factors = pd.read_excel('../data/factor_pricing_data.xlsx', sheet_name = 1)

factors = factors.set_index('Date')

factors.head()

,MKT,SMB,HML,RMW,CMA,UMD
Date,,,,,,
1980-01-31,0.0551,0.0183,0.0175,-0.0170,0.0164,0.0755
1980-02-29,-0.0122,-0.0157,0.0061,0.0004,0.0268,0.0788
1980-03-31,-0.1290,-0.0693,-0.0101,0.0146,-0.0119,-0.0955
1980-04-30,0.0397,0.0105,0.0106,-0.0210,0.0029,-0.0043
1980-05-31,0.0526,0.0211,0.0038,0.0034,-0.0031,-0.0112


In [12]:
portfolios = pd.read_excel('../data/factor_pricing_data.xlsx', sheet_name = 2)

portfolios = portfolios.set_index('Date')

portfolios.head()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,-0.0050,0.0283,0.0084,0.1024,-0.0143,0.0999,0.0354,0.0352,0.0048,0.0032,...,0.0159,0.0876,0.0463,-0.0116,0.0458,-0.0279,0.0258,0.0751,0.0299,0.0665
1980-02-29,0.0111,-0.0610,-0.0966,-0.0319,-0.0569,-0.0314,-0.0527,-0.0788,-0.0556,-0.0140,...,-0.0079,-0.0535,-0.0339,-0.0633,-0.0638,-0.0855,-0.0960,-0.0314,-0.0275,-0.0267
1980-03-31,-0.2244,-0.1116,-0.0167,-0.1464,-0.0192,-0.1281,-0.0817,-0.1278,-0.0565,-0.0664,...,-0.0821,-0.1511,-0.1106,-0.0922,-0.1443,-0.0563,-0.0883,-0.2441,-0.1245,-0.1728
1980-04-30,0.0451,0.0766,0.0232,0.0305,0.0831,-0.0521,0.0775,0.0182,0.0304,0.0113,...,0.0419,-0.0097,-0.0300,0.0351,0.0522,0.0729,0.0532,0.0997,0.0448,0.0762
1980-05-31,0.0637,0.0792,0.0457,0.0895,0.0814,0.0512,0.0324,0.0876,0.0560,0.0064,...,0.0565,0.1060,0.1147,0.0868,0.1127,0.0577,0.0557,0.1040,0.0839,0.0684


## 2. The Factors
### 1. Compare Univariate Statistics

In [3]:
def stats_dates(df, dates, annual_fac=12):
    stats_df = pd.DataFrame(data=None, index = ['Mean', 'Vol', 'Sharpe', 'VaR (.05)'])

    for d in dates:
        for col in df.columns:
            df_ = df.loc[d[0]:d[1], col]
            stats_df[col + ' ' + d[0] + '-' + d[1]] = [df_.mean()*annual_fac,
                                                       df_.std()*np.sqrt(annual_fac),
                                                       (df_.mean()/df_.std())*np.sqrt(annual_fac),
                                                       df_.quantile(.05)]

    return stats_df

def summary_stats(df, annual_fac=12):
    ss_df = (df.mean() * annual_fac).to_frame('Mean')
    ss_df['Vol'] = df.std() * np.sqrt(annual_fac)
    ss_df['Sharpe'] = ss_df['Mean'] / ss_df['Vol']

    return round(ss_df, 4)

In [4]:
summary_stats(factors)

,Mean,Vol,Sharpe
MKT,0.0831,0.1567,0.5305
SMB,0.0122,0.1005,0.1211
HML,0.0275,0.1088,0.2529
RMW,0.0448,0.0834,0.5376
CMA,0.0333,0.0715,0.4652
UMD,0.0655,0.1545,0.4241


### 2.
#### (a):
- Each factor does have a greater than 0 expected excess return. Therefore, we may say that each factor does have premium

#### (b):

In [6]:
stats_dates(factors, [['2015','2021']])

,MKT 2015-2021,SMB 2015-2021,HML 2015-2021,RMW 2015-2021,CMA 2015-2021,UMD 2015-2021
Mean,0.1427,-0.0077,-0.0485,0.0433,-0.0169,0.0159
Vol,0.1511,0.0996,0.1192,0.0695,0.0633,0.1399
Sharpe,0.9445,-0.0775,-0.4066,0.6227,-0.2677,0.1135
VaR (.05),-0.0680,-0.0432,-0.0467,-0.0225,-0.0251,-0.0671


- MKT, RMW, and UMD are the only factors with positive risk premium in the set period. HML factor noticeably under perferomed with the expected return of approximately -4.85%

### 3.

In [8]:
factors.corr()

,MKT,SMB,HML,RMW,CMA,UMD
MKT,1.0000,0.2263,-0.2221,-0.2554,-0.3819,-0.1677
SMB,0.2263,1.0000,-0.0721,-0.4143,-0.0642,-0.0304
HML,-0.2221,-0.0721,1.0000,0.2295,0.6725,-0.2349
RMW,-0.2554,-0.4143,0.2295,1.0000,0.1155,0.0753
CMA,-0.3819,-0.0642,0.6725,0.1155,1.0000,-0.0122
UMD,-0.1677,-0.0304,-0.2349,0.0753,-0.0122,1.0000


#### (a)
- Yes, correlations between factors are kept relatively small. The largest correlation is 0.6725, which is much higher than the other correlations.
#### (b)
- Yes, HML is highly correlated to CMA (this is the 0.6725 correlation).
### 4.

In [9]:
def compute_tangency(df_tilde, diagonalize_Sigma=False):

    Sigma = df_tilde.cov()

    # N is the number of assets

    N = Sigma.shape[0]

    Sigma_adj = Sigma.copy()

    if diagonalize_Sigma:

        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))



    mu_tilde = df_tilde.mean()

    Sigma_inv = np.linalg.inv(Sigma_adj)

    weights = Sigma_inv @ mu_tilde / (np.ones(N) @ Sigma_inv @ mu_tilde)

    # For convenience, I'll wrap the solution back into a pandas.Series object.

    omega_tangency = pd.Series(weights, index=mu_tilde.index)

    return omega_tangency, mu_tilde, Sigma_adj



omega_tangency, mu_tilde, Sigma = compute_tangency(factors)

omega_tangency.to_frame('Tangency Weights')

,Tangency Weights
MKT,0.2011
SMB,0.0816
HML,-0.0470
RMW,0.2884
CMA,0.3774
UMD,0.0986


#### (a):
- CMA, RMW, and the MKT seem like the most important factors as they have the largest weights. SMB, HML, and UMD have lower weights so we could say that they seem less important.
#### (b):

In [10]:
summary_stats(factors)

,Mean,Vol,Sharpe
MKT,0.0831,0.1567,0.5305
SMB,0.0122,0.1005,0.1211
HML,0.0275,0.1088,0.2529
RMW,0.0448,0.0834,0.5376
CMA,0.0333,0.0715,0.4652
UMD,0.0655,0.1545,0.4241


- Yes, CMA has one of the lower mean returns but the highest allocation.

In [11]:
omega_tangency2, mu_tilde2, Sigma2 = compute_tangency(factors[['MKT','SMB','HML','UMD']])

omega_tangency2.to_frame('Tangency Weights')

,Tangency Weights
MKT,0.3314
SMB,0.0061
HML,0.3622
UMD,0.3003


- HML has the highest tangency weight once we remove CMA. This makes sense as CMA had the largest weight before, and is quite correlated to HML. SMB has a very small weight now.

#### Conclusion of styles:
- The importance of these styles seems to be determined by the correlation between the factors.

## 3. Testing Modern LPMs

In [13]:
CAPM =  ['MKT']
FF_3F = ['MKT','SMB','HML']
FF_5F = ['MKT','SMB','HML','RMW','CMA']
AQR = ['MKT','HML','RMW','UMD']

In [14]:
def ts_test(df, factor_df, factors, test, annualization=12):
    res = pd.DataFrame(data = None, index = df.columns, columns = [test + r' $\alpha$', test + r' $R^{2}$'])

    for port in df.columns:
        y = df[port]
        X = sm.add_constant(factor_df[factors])
        model = sm.OLS(y, X).fit()
        res.loc[port] = [model.params[0] * annualization, model.rsquared]

    return res

### 1.
#### (a):

In [15]:
AQR_test = ts_test(portfolios, factors, AQR, 'AQR')

AQR_test

,AQR $\alpha$,AQR $R^{2}$
Agric,0.0156,0.3302
Food,0.0152,0.4681
Soda,0.0238,0.3098
Beer,0.0268,0.4248
Smoke,0.0399,0.2575
Toys,-0.0277,0.5033
Fun,0.0271,0.6156
Books,-0.0292,0.6886
Hshld,-0.0009,0.5681
Clths,-0.0014,0.6185


#### (b):

In [16]:
print('AQR MAE: ' + str(round(AQR_test[r'AQR $\alpha$'].abs().mean(), 4)))

AQR MAE: 0.0235


- If the pricing model worked the MAE should be close to 0 because the expected value of an excess return should be able to be described exactly by the betas and expected return of factors--leaving the expected value of alpha as 0. The AQR MAE is still pretty small. However, if the MAE takes a statistically significant non-zero value, it does not support the pricing model

### 2.

In [17]:
factor_tests = ts_test(portfolios, factors, CAPM, 'CAPM').join(ts_test(portfolios, factors, FF_3F, 'Fama-French 3F'))\
                                                         .join(ts_test(portfolios, factors, FF_5F, 'Fama-French 5F'))

factors_MAE = factor_tests[[r'CAPM $\alpha$',
                            r'Fama-French 3F $\alpha$',
                            r'Fama-French 5F $\alpha$']].abs().mean().to_frame('MAE')

factors_MAE.index = ['CAPM','Fama-French 3F','Fama-French 5F']
factors_MAE.loc['AQR'] = AQR_test[r'AQR $\alpha$'].abs().mean()
factors_MAE

,MAE
CAPM,0.0215
Fama-French 3F,0.0254
Fama-French 5F,0.0325
AQR,0.0235


- CAPM fits the best as it has the lowest MAE.
### 3.
- The market factor seems very important for pricing as all models include it and the CAPM performs the best. I think Fama and French should consider using the momentum factor as AQR uses it and their model performs better in terms of MAE.
### 4.

In [18]:
factors_r2 = factor_tests[[r'CAPM $R^{2}$',
                            r'Fama-French 3F $R^{2}$',
                            r'Fama-French 5F $R^{2}$']].mean().to_frame(r'$R^{2}$')

factors_r2.index = ['CAPM','Fama-French 3F','Fama-French 5F']
factors_r2.loc['AQR'] = AQR_test[r'AQR $R^{2}$'].mean()
factors_r2

,$R^{2}$
CAPM,0.5275
Fama-French 3F,0.5711
Fama-French 5F,0.5964
AQR,0.5757


- The models do not lead to a high $R^{2}$ stats. Therefore, they would not be a good linear factor decomposition of the assets

### 5.

In [22]:
def ts_betas(df, factor_df, factors, intercept=False):
    if intercept == True:
        res = pd.DataFrame(data = None, index = df.columns, columns = ['alpha'])
        res[factors] = None
    else:
        res = pd.DataFrame(data = None, index = df.columns, columns = factors)

    for port in df.columns:
        y = df[port]
        if intercept == True:
            X = sm.add_constant(factor_df[factors])
        else:
            X = factor_df[factors]
        model = sm.OLS(y, X).fit()
        res.loc[port] = model.params

    return res

def cross_section(df, factor_df, factors, ts_int=True, annualization=12):
    betas = ts_betas(df, factor_df, factors, intercept=ts_int)
    res = pd.DataFrame(data = None, index = betas.index, columns = factors)
    res['Predicted'] = None
    res['Actual'] = None

    for port in res.index:
        res.loc[port, factors] = betas.loc[port]
        prem = (betas.loc[port] * factor_df[factors]).sum(axis=1).mean() * annualization
        res.loc[port,['Predicted','Actual']] = prem, df[port].mean() * annualization

    return res

def cross_premia(df_cs, factors):
    y = df_cs['Actual'].astype(float)
    X = df_cs[factors].astype(float)

    return sm.OLS(y,X).fit().params.to_frame('Cross-Sectional Premia')

def cross_premia_mae(df_cs, factors, model):
    y = df_cs['Actual'].astype(float)
    X = df_cs[factors].astype(float)

    print(model + ' MAE: ' + str(round(sm.OLS(y,X).fit().resid.abs().mean(), 4)))
    return

In [23]:
CAPM_cs = cross_section(portfolios, factors, CAPM, ts_int=True)
FF_3F_cs = cross_section(portfolios, factors, FF_3F, ts_int=True)
FF_5F_cs = cross_section(portfolios, factors, FF_5F, ts_int=True)
AQR_cs = cross_section(portfolios, factors, AQR, ts_int=True)

#### (a):

In [24]:
(factors.mean()*12).to_frame('Time Series Premia')

,Time Series Premia
MKT,0.0831
SMB,0.0122
HML,0.0275
RMW,0.0448
CMA,0.0333
UMD,0.0655


- Fama-French 3 Factor Premia:

In [25]:
cross_premia(FF_3F_cs, FF_3F)

,Cross-Sectional Premia
MKT,0.1010
SMB,-0.0659
HML,-0.0173


- Fama-French 5 Factor Premia:

In [26]:
cross_premia(FF_5F_cs, FF_5F)

,Cross-Sectional Premia
MKT,0.0948
SMB,-0.0587
HML,-0.0354
RMW,0.0368
CMA,-0.0154


- AQR Premia:

In [27]:
cross_premia(AQR_cs, AQR)

,Cross-Sectional Premia
MKT,0.0866
HML,-0.0409
RMW,0.0455
UMD,0.0553


- The market and RMW factors seem to report similar premia in the cross-sectional analysis when compared to the time-series analysis. However, the other factors seems to vary a lot from the time-series approach.

#### (b):

In [28]:
cross_premia_mae(CAPM_cs, CAPM, 'CAPM')

CAPM MAE: 0.0214


In [29]:
cross_premia_mae(FF_3F_cs, FF_3F, 'FF 3 Factor')

FF 3 Factor MAE: 0.0161


In [30]:
cross_premia_mae(FF_5F_cs, FF_5F, 'FF 5 Factor')

FF 5 Factor MAE: 0.0136


In [31]:
cross_premia_mae(AQR_cs, AQR, 'AQR')

AQR MAE: 0.0172


- The CAPM model seems to report similar MAE to the time-series alphas. However, all the other models report lower MAE when compared to the time-series alphas.